# 第六章：Layer 2 多帧集成分析 — 从噪声到统计

## 为什么需要多帧平均？

在第五章中，我们对第一帧进行了单帧分析，得到的密度曲线**噪声很大**：
- 只有有限个水分子（几十到几百个）
- 热噪声导致每帧的分布都略有不同
- 单帧结果对初始构型高度敏感

**解决方案**：**系综平均（Ensemble Average）**——对所有帧的结果取平均，  
使统计波动相互抵消，得到平滑、物理可靠的曲线。

> **统计力学基础**：平衡态性质可以用时间平均代替系综平均（遍历性假设）。  
> MD 轨迹中，不同时刻的帧代表系综中不同的微观状态。

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
sys.path.insert(0, str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / "data_example" / "potential"
OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ase.io

from src.structure.Analysis.WaterAnalysis._common import (
    _parse_abc_from_md_inp,
    _iter_trajectory,
    _single_frame_density_and_orientation,
    _compute_density_orientation_ensemble,
)

md_inp_path = DATA_DIR / "md.inp"
xyz_path    = DATA_DIR / "md-pos-1.xyz"

a_A, b_A, c_A = _parse_abc_from_md_inp(md_inp_path)
print(f"晶胞参数: a={a_A:.3f}, b={b_A:.3f}, c={c_A:.3f} Å")

---

## 6.1 单帧 vs 平均：直观对比

先计算前几帧的单帧密度，叠加展示，再对比全部平均的结果。

> ⚠️ 下面的 cell 计算多帧数据，**每帧约需 1~3 秒**，共计算 10 帧（约 10~30 秒）

In [ ]:
# 计算前 N 帧的单帧密度
N_demo = 10  # 用于演示的帧数

per_frame_rho = []    # 每帧的密度曲线
per_frame_centers = [] # 每帧的 bin 中心（Å）
per_frame_path = []    # 每帧的路径长度（Å）

print(f"正在计算前 {N_demo} 帧...")
for frame_idx, atoms in enumerate(_iter_trajectory(xyz_path, a_A, b_A, c_A)):
    if frame_idx >= N_demo:
        break
    
    centers_A, rho, orient, path_A = _single_frame_density_and_orientation(
        atoms,
        start_interface="low_c",
        dz_A=0.1,
        metal_symbols=None,
        compute_orientation=False,  # 只计算密度，跳过取向（更快）
    )
    per_frame_rho.append(rho)
    per_frame_centers.append(centers_A)
    per_frame_path.append(path_A)
    print(f"  帧 {frame_idx}: {len(rho)} bins, 路径长度 {path_A:.3f} Å")

print(f"\n完成！计算了 {len(per_frame_rho)} 帧")

In [ ]:
# 插值到统一的归一化网格，然后取平均
# （各帧路径长度可能微小不同，需要先归一化再平均）
mean_path_A = float(np.mean(per_frame_path))

# 建立统一的归一化网格（分数坐标 u = 距离/路径长度）
n_bins_common = max(1, int(np.ceil(mean_path_A / 0.1)))
common_edges_u = np.linspace(0.0, 1.0, n_bins_common + 1)
common_centers_u = 0.5 * (common_edges_u[:-1] + common_edges_u[1:])

# 对每帧插值到统一网格
interp_list = []
for centers_A, rho, path_A in zip(per_frame_centers, per_frame_rho, per_frame_path):
    centers_u = centers_A / path_A  # 归一化：分数坐标
    rho_interp = np.interp(common_centers_u, centers_u, rho, left=0.0, right=0.0)
    interp_list.append(rho_interp)

# 平均
rho_10frame_avg = np.mean(np.array(interp_list), axis=0)

# 物理距离（Å）
common_dist_A = common_centers_u * mean_path_A

print(f"统一网格：{n_bins_common} bins，平均路径 {mean_path_A:.3f} Å")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

# 灰色细线：每帧单独的密度曲线
for i, (centers_A, rho, path_A) in enumerate(zip(per_frame_centers, per_frame_rho, per_frame_path)):
    if i == 0:
        ax.plot(centers_A, rho, color='gray', linewidth=0.6, alpha=0.6, label=f'单帧曲线（共 {N_demo} 帧）')
    else:
        ax.plot(centers_A, rho, color='gray', linewidth=0.6, alpha=0.6)

# 红色粗线：10 帧平均
ax.plot(common_dist_A, rho_10frame_avg, color='tab:red', linewidth=2.5,
        label=f'{N_demo} 帧平均', zorder=5)

# 体相水参考
ax.axhline(y=1.0, color='black', linestyle='--', linewidth=1, alpha=0.5, label='体相水 1.0 g/cm³')

ax.set_xlabel('距界面距离 (Å)', fontsize=12)
ax.set_ylabel('水密度 (g/cm³)', fontsize=12)
ax.set_title(f'单帧 vs {N_demo} 帧平均：密度分布对比', fontsize=13)
ax.legend(fontsize=10)
ax.set_xlim(0, None)

plt.tight_layout()
plt.show()

print("观察：")
print("  · 单帧曲线（灰色）噪声大，随帧波动明显")
print("  · 多帧平均（红色）更加平滑，峰值位置更加清晰")

---

## 6.2 归一化网格的必要性

### 为什么不能直接平均？

不同帧的晶胞长度可能略有差异（NVT 系综温度波动会导致微小的结构变化），  
或者界面位置有微小变化，导致"从界面到中点"的路径长度 `path_length_A` 略有不同。

**直接平均**会导致：不同帧的 bin 对应的物理位置不完全相同，平均后曲线失真。

**正确做法**：先将每帧的坐标归一化（$u = \text{距离} / \text{路径长度}$），  
在统一的 $u \in [0, 1]$ 网格上插值，然后平均，最后再转回物理距离（用平均路径长度）。

In [ ]:
# 展示各帧路径长度的变化
print("前 10 帧的路径长度：")
for i, path in enumerate(per_frame_path):
    bar = '█' * int(path * 5 / per_frame_path[0]) 
    print(f"  帧 {i:2d}: {path:.4f} Å  {bar}")

print(f"\n路径长度统计：")
print(f"  最小值: {min(per_frame_path):.4f} Å")
print(f"  最大值: {max(per_frame_path):.4f} Å")
print(f"  标准差: {np.std(per_frame_path):.4f} Å")
print(f"  变化幅度: {max(per_frame_path) - min(per_frame_path):.4f} Å")
print()
print("归一化策略：")
print("  u = 距离 / 路径长度，范围 [0, 1]")
print("  对所有帧插值到 u 的统一网格")
print("  最后乘以平均路径长度，转回物理距离")

---

## 6.3 `_common.py` 工作流程

Layer 2 的核心函数 `_compute_density_orientation_ensemble` 按以下流程工作：

```
输入: xyz_path, md_inp_path
  │
  ├── 解析晶胞参数 (a, b, c)
  │     ↓
  └── 循环遍历每一帧:
        │
        ├── 设置晶胞 (a, b, c)，开启 PBC
        │
        ├── 检测界面层 → low_c, high_c
        │
        ├── 检测水分子拓扑 → (n_water, 3) 索引
        │
        ├── 计算密度分布 → rho(z), 形状 (nbins,)
        │
        ├── 计算取向加权密度 → orient(z), 形状 (nbins,)
        │
        └── 归一化到 [0,1] 网格，存储
  │
  └── 所有帧插值到统一网格，等权重平均
        ↓
输出: common_centers_u, mean_path_A, rho_ensemble, orient_ensemble
```

---

## 6.4 帧数对结果的影响

先用已有的 10 帧平均结果，与完整 274 帧结果对比。  
（如果第一章已经运行，直接读取 CSV 文件）

> ⚠️ 计算全部 274 帧需要约 2~5 分钟。如果已经在第一章运行过，下面直接读取结果。

In [ ]:
密度CSV路径 = OUTPUT_DIR / "water_mass_density_z_distribution_analysis.csv"

if 密度CSV路径.exists():
    # 直接读取已有结果
    数据 = np.genfromtxt(密度CSV路径, delimiter=',', skip_header=1)
    dist_full = 数据[:, 1]    # 距离（Å）
    rho_full  = 数据[:, 2]    # 密度（g/cm³）
    print(f"从文件读取 274 帧结果：{len(dist_full)} 个数据点")
else:
    # 重新计算（需要较长时间）
    print("未找到预计算结果，正在重新计算 274 帧（约 2~5 分钟）...")
    from src.structure.Analysis import plot_water_three_panel_analysis
    plot_water_three_panel_analysis(
        xyz_path=xyz_path,
        md_inp_path=md_inp_path,
        output_dir=OUTPUT_DIR,
    )
    数据 = np.genfromtxt(密度CSV路径, delimiter=',', skip_header=1)
    dist_full = 数据[:, 1]
    rho_full  = 数据[:, 2]
    print(f"计算完成：{len(dist_full)} 个数据点")

In [ ]:
# 对比：单帧 vs 10帧 vs 274帧
fig, ax = plt.subplots(figsize=(10, 5))

# 灰色：单帧曲线（前3帧）
for i, (centers_A, rho, path_A) in enumerate(zip(per_frame_centers[:3], per_frame_rho[:3], per_frame_path[:3])):
    label = '单帧曲线（第0~2帧）' if i == 0 else '_'
    ax.plot(centers_A, rho, color='gray', linewidth=0.8, alpha=0.6, label=label)

# 红色：10帧平均
ax.plot(common_dist_A, rho_10frame_avg, color='tab:red', linewidth=2,
        label=f'{N_demo} 帧平均', zorder=4)

# 蓝色：274帧平均（最终结果）
ax.plot(dist_full, rho_full, color='tab:blue', linewidth=2.5,
        label='274 帧平均（最终结果）', zorder=5)

ax.axhline(y=1.0, color='black', linestyle='--', linewidth=1, alpha=0.5, label='体相水 1.0 g/cm³')

ax.set_xlabel('距界面距离 (Å)', fontsize=12)
ax.set_ylabel('水密度 (g/cm³)', fontsize=12)
ax.set_title('帧数对密度曲线平滑度的影响', fontsize=13)
ax.legend(fontsize=10)
ax.set_xlim(0, None)

plt.tight_layout()
plt.show()

print("观察：")
print("  · 灰色（单帧）: 噪声大，难以辨别物理结构")
print("  · 红色（10帧）: 已经可以看出峰的大致位置")
print("  · 蓝色（274帧）: 峰值清晰，曲线平滑")

---

## 6.5 吸附层自动检测

`detect_adsorbed_layer_range_from_density_profile` 函数自动识别密度曲线中的**第一个峰**，
并确定吸附层的起始和结束位置。

**算法步骤**：
1. 找到密度最大值的位置（主峰）
2. 向左找：第一个近零点（密度 < 5% 峰值）
3. 向右找：主峰之后的第一个局部最小值
4. [左端, 右端] 即为吸附层范围

In [ ]:
from src.structure.Analysis import detect_adsorbed_layer_range_from_density_profile

# 自动检测吸附层范围
d_start_A, d_end_A, d_peak_A = detect_adsorbed_layer_range_from_density_profile(
    dist_full, rho_full
)

print(f"自动检测到的吸附层范围：")
print(f"  起始位置: {d_start_A:.3f} Å")
print(f"  主峰位置: {d_peak_A:.3f} Å  ←  密度最高点")
print(f"  结束位置: {d_end_A:.3f} Å")
print(f"  吸附层宽度: {d_end_A - d_start_A:.3f} Å")

In [ ]:
# 可视化：在密度曲线上标注吸附层范围
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(dist_full, rho_full, color='tab:blue', linewidth=2, label='水密度（274帧平均）')

# 用阴影标注吸附层范围
吸附层掩码 = (dist_full >= d_start_A) & (dist_full <= d_end_A)
ax.fill_between(dist_full, 0, rho_full,
                where=吸附层掩码,
                alpha=0.4, color='tab:green',
                label=f'吸附层 ({d_start_A:.2f} ~ {d_end_A:.2f} Å)')

# 标注主峰
ax.axvline(x=d_peak_A, color='red', linestyle='--', linewidth=1.5,
           label=f'主峰 ({d_peak_A:.2f} Å)')

# 标注界面（x=0 是金属表面）
ax.axvline(x=0, color='black', linestyle='-', linewidth=2, alpha=0.5)
ax.text(0.1, ax.get_ylim()[1] * 0.9 if ax.get_ylim()[1] > 0 else 3, '← 金属表面', fontsize=9)

ax.axhline(y=1.0, color='gray', linestyle=':', linewidth=1, label='体相水 1.0 g/cm³')

ax.set_xlabel('距界面距离 (Å)', fontsize=12)
ax.set_ylabel('水密度 (g/cm³)', fontsize=12)
ax.set_title('自动检测的吸附层范围（绿色阴影区域）', fontsize=13)
ax.legend(fontsize=10)
ax.set_xlim(0, None)

plt.tight_layout()
plt.show()

---

## 6.6 输出文件解读

完整运行 `plot_water_three_panel_analysis` 会在 `tutorial/output/` 中生成多个文件。
逐个查看和理解这些文件。

In [ ]:
# 文件1：水密度剖面 CSV
print("=== water_mass_density_z_distribution_analysis.csv ===")
密度数据 = pd.read_csv(OUTPUT_DIR / "water_mass_density_z_distribution_analysis.csv")
print(f"列名: {list(密度数据.columns)}")
print(f"行数: {len(密度数据)}")
print()
print("前 5 行：")
print(密度数据.head().to_string())
print()
print("各列含义：")
print("  path_fraction_center: 归一化路径坐标（0=界面, 1=中点）")
print("  distance_A: 物理距离（Å）")
print("  rho_ensemble_avg_g_cm3: 系综平均质量密度（g/cm³）")

In [ ]:
# 文件2：取向加权密度剖面 CSV
print("=== water_orientation_weighted_density_z_distribution_analysis.csv ===")
取向CSV路径 = OUTPUT_DIR / "water_orientation_weighted_density_z_distribution_analysis.csv"
if 取向CSV路径.exists():
    取向数据 = pd.read_csv(取向CSV路径)
    print(f"列名: {list(取向数据.columns)}")
    print(f"行数: {len(取向数据)}")
    print("\n前 5 行：")
    print(取向数据.head().to_string())
else:
    print("文件不存在，请先运行第一章")

In [ ]:
# 文件3：吸附层剖面 CSV
print("=== adsorbed_water_orientation_profile.csv ===")
吸附CSV路径 = OUTPUT_DIR / "adsorbed_water_orientation_profile.csv"
if 吸附CSV路径.exists():
    吸附数据 = pd.read_csv(吸附CSV路径)
    print(f"列名: {list(吸附数据.columns)}")
    print(f"行数: {len(吸附数据)}")
    print("\n前 5 行：")
    print(吸附数据.head().to_string())
    print()
    print("is_adsorbed_layer_bin = 1 的行：")
    吸附层行 = 吸附数据[吸附数据['is_adsorbed_layer_bin'] == 1]
    print(f"  {len(吸附层行)} 个 bin 在吸附层中")
    print(f"  距离范围: {吸附层行['distance_A'].min():.3f} ~ {吸附层行['distance_A'].max():.3f} Å")
else:
    print("文件不存在，请先运行第一章")

In [ ]:
# 文件4：吸附层范围 TXT
range_txt路径 = OUTPUT_DIR / "adsorbed_water_layer_range.txt"
print("=== adsorbed_water_layer_range.txt ===")
if range_txt路径.exists():
    内容 = range_txt路径.read_text()
    print(内容)
    print()
    print("各字段含义：")
    print("  adsorbed_layer_start_A: 吸附层起始距离（Å）")
    print("  adsorbed_layer_end_A:   吸附层结束距离（Å）")
    print("  main_peak_distance_A:   密度主峰位置（Å）")
    print("  near_zero_ratio:        近零阈值比例（5% 峰值密度）")
    print("  smoothing_window_bins:  平滑窗口大小（bins）")
else:
    print("文件不存在")

In [ ]:
# 文件5：吸附层角度分布 CSV
theta_csv路径 = OUTPUT_DIR / "adsorbed_water_theta_distribution_0_180.csv"
print("=== adsorbed_water_theta_distribution_0_180.csv ===")
if theta_csv路径.exists():
    theta数据 = pd.read_csv(theta_csv路径)
    print(f"列名: {list(theta数据.columns)}")
    print(f"行数: {len(theta数据)} (每行对应一个 5° 角度区间)")
    print("\n前 5 行：")
    print(theta数据.head().to_string())
else:
    print("文件不存在")

In [ ]:
# 综合可视化：展示所有 CSV 中的关键曲线
fig, axes = plt.subplots(3, 1, figsize=(10, 12))

# 上图：密度剖面
ax1 = axes[0]
ax1.plot(密度数据['distance_A'], 密度数据['rho_ensemble_avg_g_cm3'],
         color='tab:blue', linewidth=2, label='水密度')
if 吸附CSV路径.exists():
    in_ad = 吸附数据['is_adsorbed_layer_bin'] == 1
    ax1.fill_between(吸附数据['distance_A'], 0, 吸附数据['rho_ensemble_avg_g_cm3'],
                     where=in_ad, alpha=0.4, color='tab:green', label='吸附层范围')
ax1.axhline(y=1.0, color='gray', linestyle='--', linewidth=1)
ax1.set_xlabel('距界面距离 (Å)', fontsize=11)
ax1.set_ylabel('密度 (g/cm³)', fontsize=11)
ax1.set_title('Panel 1: 水的质量密度分布', fontsize=12)
ax1.legend(fontsize=10)
ax1.set_xlim(0, None)

# 中图：取向加权密度
ax2 = axes[1]
if 取向CSV路径.exists():
    ax2.plot(取向数据['distance_A'], 取向数据['orientation_ensemble_avg_g_cm3'],
             color='tab:orange', linewidth=2, label='取向加权密度')
    ax2.axhline(y=0, color='gray', linestyle='--', linewidth=1)
ax2.set_xlabel('距界面距离 (Å)', fontsize=11)
ax2.set_ylabel('ρ·cosθ (g/cm³)', fontsize=11)
ax2.set_title('Panel 2: 取向加权密度分布', fontsize=12)
ax2.legend(fontsize=10)
ax2.set_xlim(0, None)

# 下图：吸附层角度 PDF
ax3 = axes[2]
if theta_csv路径.exists():
    ax3.plot(theta数据['theta_degree'], theta数据['pdf_degree_inv'],
             color='tab:green', linewidth=2, label='吸附层 θ 分布')
    ax3.fill_between(theta数据['theta_degree'], 0, theta数据['pdf_degree_inv'],
                     alpha=0.3, color='tab:green')
ax3.set_xlabel('取向角 θ (度)', fontsize=11)
ax3.set_ylabel('概率密度 (1/度)', fontsize=11)
ax3.set_title('Panel 3: 吸附层水分子取向角分布', fontsize=12)
ax3.set_xlim(0, 180)
ax3.legend(fontsize=10)

plt.suptitle('综合三联图（从 CSV 文件重建）', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

---

## 6.7 参数调节实验

### 实验1：改变 `dz_A`（bin 宽度）的影响

`dz_A` 越小 → bin 越多 → 曲线分辨率越高，但噪声也越大  
`dz_A` 越大 → bin 越少 → 曲线越平滑，但细节丢失

In [ ]:
# 用前 5 帧的数据对比不同 dz_A 的效果
dz_values = [0.05, 0.1, 0.2]  # Å
颜色列表 = ['tab:purple', 'tab:blue', 'tab:green']

fig, ax = plt.subplots(figsize=(10, 5))

for dz, 色 in zip(dz_values, 颜色列表):
    # 对前5帧用不同 dz_A 计算密度
    frame_rho_list = []
    frame_centers_list = []
    frame_path_list = []
    
    for frame_idx, atoms in enumerate(_iter_trajectory(xyz_path, a_A, b_A, c_A)):
        if frame_idx >= 5:
            break
        centers_A, rho, orient, path_A = _single_frame_density_and_orientation(
            atoms, start_interface="low_c", dz_A=dz,
            metal_symbols=None, compute_orientation=False,
        )
        frame_rho_list.append(rho)
        frame_centers_list.append(centers_A)
        frame_path_list.append(path_A)
    
    # 平均
    mean_path = float(np.mean(frame_path_list))
    n_bins = max(1, int(np.ceil(mean_path / dz)))
    edges_u = np.linspace(0, 1, n_bins + 1)
    centers_u = 0.5 * (edges_u[:-1] + edges_u[1:])
    
    interp = [np.interp(centers_u, c/p, r, left=0, right=0)
              for c, r, p in zip(frame_centers_list, frame_rho_list, frame_path_list)]
    rho_avg = np.mean(interp, axis=0)
    dist_avg = centers_u * mean_path
    
    ax.plot(dist_avg, rho_avg, color=色, linewidth=1.5, label=f'dz_A = {dz} Å', alpha=0.8)

ax.axhline(y=1.0, color='gray', linestyle='--', linewidth=1)
ax.set_xlabel('距界面距离 (Å)', fontsize=12)
ax.set_ylabel('水密度 (g/cm³)', fontsize=12)
ax.set_title('不同 dz_A 参数对密度曲线的影响（5帧平均）', fontsize=13)
ax.legend(fontsize=10)
ax.set_xlim(0, None)

plt.tight_layout()
plt.show()

print("结论：")
print("  dz_A=0.05 Å: 最高分辨率，噪声最大")
print("  dz_A=0.10 Å: 默认值，平衡分辨率和噪声")
print("  dz_A=0.20 Å: 最平滑，但峰值位置分辨率下降")

### 实验2：`start_interface` 参数

`start_interface` 决定从哪个界面出发分析：
- `"low_c"`：从 z 较小的界面出发（默认）
- `"high_c"`：从 z 较大的界面出发

理论上两个界面给出的结果应该相同（因为金属板两侧对称），  
实际因晶格和水分子的不同可能略有差异。

In [ ]:
# 对比 low_c 和 high_c 两个界面的密度曲线（用5帧做快速对比）
fig, ax = plt.subplots(figsize=(10, 5))

for interface, 色 in [('low_c', 'tab:blue'), ('high_c', 'tab:orange')]:
    frame_rho_list = []
    frame_centers_list = []
    frame_path_list = []
    
    for frame_idx, atoms in enumerate(_iter_trajectory(xyz_path, a_A, b_A, c_A)):
        if frame_idx >= 5:
            break
        centers_A, rho, orient, path_A = _single_frame_density_and_orientation(
            atoms, start_interface=interface, dz_A=0.1,
            metal_symbols=None, compute_orientation=False,
        )
        frame_rho_list.append(rho)
        frame_centers_list.append(centers_A)
        frame_path_list.append(path_A)
    
    mean_path = float(np.mean(frame_path_list))
    n_bins = max(1, int(np.ceil(mean_path / 0.1)))
    edges_u = np.linspace(0, 1, n_bins + 1)
    centers_u = 0.5 * (edges_u[:-1] + edges_u[1:])
    
    interp = [np.interp(centers_u, c/p, r, left=0, right=0)
              for c, r, p in zip(frame_centers_list, frame_rho_list, frame_path_list)]
    rho_avg = np.mean(interp, axis=0)
    dist_avg = centers_u * mean_path
    
    ax.plot(dist_avg, rho_avg, color=色, linewidth=2, label=f'start_interface="{interface}"', alpha=0.8)

ax.axhline(y=1.0, color='gray', linestyle='--', linewidth=1)
ax.set_xlabel('距界面距离 (Å)', fontsize=12)
ax.set_ylabel('水密度 (g/cm³)', fontsize=12)
ax.set_title('不同起始界面对密度曲线的影响（5帧平均）', fontsize=13)
ax.legend(fontsize=10)
ax.set_xlim(0, None)

plt.tight_layout()
plt.show()

print("\n两条曲线形状相近说明：")
print("  · 金属板两个界面的水结构大致对称")
print("  · 微小差异来自有限帧数和两侧环境的统计涨落")

---

## 本章小结 & 全教程总结

### 第六章学到的内容

| 概念 | 说明 |
|------|------|
| 系综平均 | 多帧结果取均值，减小统计涨落 |
| 归一化网格 | 先归一化路径坐标，再插值平均，避免路径长度差异导致的错位 |
| 吸附层检测 | 找第一个密度峰的左端（近零点）和右端（局部最小值） |
| `dz_A` 参数 | 控制 bin 宽度，影响分辨率和噪声的权衡 |
| `start_interface` | 选择从哪个界面出发，两者结果应大致对称 |

### 全教程总结

```
第01章: 运行主函数 → 得到三联图（全局印象）
第02章: 学习工具库 → NumPy, Matplotlib, ASE, dataclass
第03章: 理解输入文件 → md.inp（晶胞参数）, md-pos-1.xyz（轨迹）
第04章: 金属层检测 → LayerParser, 聚类算法, 界面识别
第05章: 水分子分析 → WaterParser, 拓扑构建, θ定义, 密度公式
第06章: 集成分析 → 系综平均, 归一化网格, 吸附层, 参数调节
```

恭喜你完成了 MD-analysis 教程系列！🎉

In [ ]:
# 最终展示：完整三联图
from IPython.display import Image, display

png路径 = OUTPUT_DIR / "water_three_panel_analysis.png"
if png路径.exists():
    print("完整分析三联图：")
    display(Image(filename=str(png路径), width=700))
else:
    print("请先运行第一章（01_quickstart.ipynb）生成三联图")